Происходит загрузка следующих библиотек Python:

*   pandas - для работы с табличными данными;

*   requests - для выполнения HTTP-запросов к API;

*   json - для обработки и парсинга получаемых данных в формате JSON.


In [ ]:
import pandas as pd
import requests
import json

Взаимодействие с источником данных реализуется через официальное API портала HH.ru, обеспечивающее получение структурированной информации о вакансиях.

In [ ]:
url = 'https://api.hh.ru/vacancies'
params = {'text': 'оператор', 'per_page': '100', 'page': '1'}
src = requests.get(url, params=params)
data = src.json()

Для хранения и обработки собранной информации создается таблица данных с использованием библиотеки pandas.

In [ ]:
names = ['name', 'experience', 'employment', 'created_at', 'selary', 'address', 'type', 'schedule', 'work_schedule_by_days', 'working_hours']
df = pd.DataFrame(columns=names)

Создайте функцию sercher, которая в цикле обрабатывает данные о вакансиях. Для каждой вакансии функция должна извлекать следующие поля:

*   наименование вакансии;

*   требуемый опыт работы;

*   тип занятости;

*   дата создания объявления;

*   заработная плата;

*   адрес места работы;

*   статус вакансии (открыта/закрыта);

*   рабочий график (смена);

*   количество рабочих часов.




После сбора информации, функция должна записывать её в соответствующую таблицу.

In [ ]:
def sercher():
    for i in range(data['per_page']-1):
        val = data['items'][i]
        name = val['name']
        sel = (str(val['salary']['from']) + '-' + str(val['salary']['to']) + ' ' + str(val['salary']['currency'])) if val['salary'] != None else 'Не указано'
        exp = val['experience']['name']
        empl = val['employment']['name']
        create = val['created_at'][:10] + ' ' + val['created_at'][10:19]
        adr = val['address']['raw'] if val['address'] else 'Не указано'
        ono = val['type']['name']
        sch = val['schedule']['name']
        wsbd = val['work_schedule_by_days'][0]['name'] if val['work_schedule_by_days'] else 'Не указано'
        wh = val['working_hours'][0]['name'].replace('\xa0', ' ') if val['working_hours'] else 'Не указано'
        odata = [name, exp, empl, create, sel, adr, ono, sch, wsbd, wh]
        df.loc[val['id']] = odata

Для получения данных со всех страниц используется функция sercher, которая осуществляет последовательный цикличный перебор.


In [ ]:
for x in range(1, data['pages']):
    params = {'text': 'оператор', 'per_page': '100', 'page': x}
    print(params['page'])
    src = requests.get(url, params=params)
    data = src.json()
    sercher()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


готовый результат конвертируем в csv файл

In [ ]:
df.to_csv('Data_HHRu.csv', index=True)